In [ ]:
# Perform mann-Kendall trend test

#Open Anaconda prompt and run "pip install pymannkendall"

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pymannkendall as mk
import numpy as np

rain_data = pd.read_excel("rainfall.xlsx", sheet_name="Sheet1")  # Import Excel file

# Reshape from wide to long format
rain_data_long = rain_data.melt(id_vars="Years", var_name="Month", value_name="Rainfall")

# Create a datetime index for proper time series plotting
# Ensure months are ordered correctly
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]
rain_data_long["Month"] = pd.Categorical(rain_data_long["Month"], categories=month_order, ordered=True)

# Combine Year and Month into a datetime column
rain_data_long["Date"] = pd.to_datetime(rain_data_long["Years"].astype(str) + rain_data_long["Month"].astype(str), format="%Y%B")

# Sort by date
rain_data_long = rain_data_long.sort_values("Date")

# Plot time series
plt.figure(figsize=(16, 6))
plt.plot(rain_data_long["Date"], rain_data_long["Rainfall"], color='blue')
plt.title("Total Monthly Rainfall Time Series (1980 onwards)", fontsize=11)
plt.xlabel("Date")
plt.ylabel("Rainfall (mm)")
plt.grid(True)
years = pd.date_range(start="1980", end="2018", freq="YS")  # YS = Year Start
plt.xticks(years, [d.year for d in years], rotation=90)

# Add linear trendline
x = np.arange(len(rain_data_long))  # x as numeric index
y = rain_data_long["Rainfall"].values
coeffs = np.polyfit(x, y, deg=1)
trend = np.poly1d(coeffs)
plt.plot(rain_data_long["Date"], trend(x), color="red", linestyle="--", label="Trendline")

# Save the plot as a PNG file in the output folder
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)  # Create an output folder if it doesn’t exist
image_path = os.path.join(output_folder, "Mann Kendall trend test")
plt.savefig(image_path)
print(f"Plot saved at: {image_path}")

# Run Mann-Kendall test on the full monthly rainfall time series
result = mk.original_test(rain_data_long["Rainfall"])

# Print the results
print("Trend:", result.trend) #Increasing / Decreasing / No trend
print("p-value:", result.p) #Statistical significance
print("Tau:", result.Tau) #Kendall’s tau correlation coefficient
print("Slope:", result.slope) #Sen’s slope (trend magnitude)
print("Intercept:", result.intercept)